In [1]:
import ubiquant

In [2]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from typing import List

In [3]:
env = ubiquant.make_env()  
iter_test = env.iter_test()

In [4]:
class Net(nn.Module):
    def __init__(self,
                 dropout_mlp: float,
                 dropout_emb: float,
                 output_dims: List[int],
                 cat_dims: List[int],
                 emb_output: int,
                 categorical=False):

        super().__init__()
        self.categorical = categorical
        mlp_layers: List[nn.Module] = []
        input_dim: int = 300

        if categorical:
            cat_input_dim: int = 3774
            emb_layers: List[nn.Module] = [nn.Embedding(cat_input_dim, emb_output)]
            cat_input_dim = emb_output
            for cat_output_dim in cat_dims:
                emb_layers.append(nn.Linear(cat_input_dim, cat_output_dim))
                emb_layers.append(nn.ReLU())
                emb_layers.append(nn.Dropout(dropout_emb))
                cat_input_dim = cat_output_dim

            input_dim += cat_output_dim

            for output_dim in output_dims:
                mlp_layers.append(nn.Linear(input_dim, output_dim))
                mlp_layers.append(nn.ReLU())
                mlp_layers.append(nn.Dropout(dropout_mlp))
                input_dim = output_dim
        else:
            for output_dim in output_dims:
                mlp_layers.append(nn.Linear(input_dim, output_dim))
                mlp_layers.append(nn.ReLU())
                mlp_layers.append(nn.Dropout(dropout_mlp))
                input_dim = output_dim

        mlp_layers.append(nn.Linear(input_dim, 1))

        if self.categorical:
            self.emb_nn: nn.Module = nn.Sequential(*emb_layers)
        self.mlp_nn: nn.Module = nn.Sequential(*mlp_layers)

    def forward(self, x_cont, x_cat):
        if self.categorical:
            x_cat = self.emb_nn(x_cat)
            concat = torch.cat([x_cat, x_cont], 1)
            output = self.mlp_nn(concat)
        else:
            output = self.mlp_nn(x_cont)
        return output


class UbiquantModel(pl.LightningModule):
    def __init__(self,
                 dropout_mlp: float,
                 dropout_emb: float,
                 output_dims: List[int],
                 emb_dims: List[int],
                 emb_output: int,
                 l_rate: float,
                 categorical: bool):
        super().__init__()
        self.model = Net(dropout_mlp,
                         dropout_emb,
                         output_dims,
                         emb_dims,
                         emb_output,
                         categorical=categorical)
        self.l_rate = l_rate
        self.categorical = categorical

    def forward(self, x_cont, x_cat):
        return self.model(x_cont, x_cat)

    def training_step(self, batch, batch_idx):
        if self.categorical:
            x_cont, x_cat, y = batch
            logits = self.forward(x_cont, x_cat)
        else:
            x_cont, _, y = batch
            logits = self.forward(x_cont, [])
        loss = pearson_loss(logits, y)
        logs = {'loss': loss}
        return {'loss': loss, 'log': logs}

In [5]:
model = UbiquantModel.load_from_checkpoint('../input/ubiquantmodels/fold-4-ubiquant-mlp-epoch14-val_loss0.00.ckpt',
                                           dropout_mlp=0.4217199217221381,
                                           dropout_emb=0.4250209544891712,
                                           output_dims=[508, 405],
                                           emb_dims=[245, 238, 230],
                                           emb_output=56,
                                           l_rate=0.00026840511349794486,
                                           categorical=False)

In [6]:
model.eval()

UbiquantModel(
  (model): Net(
    (mlp_nn): Sequential(
      (0): Linear(in_features=300, out_features=508, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.4217199217221381, inplace=False)
      (3): Linear(in_features=508, out_features=405, bias=True)
      (4): ReLU()
      (5): Dropout(p=0.4217199217221381, inplace=False)
      (6): Linear(in_features=405, out_features=1, bias=True)
    )
  )
)

In [7]:
for (test_df, sub_df) in iter_test:
    data = test_df.drop(['row_id', 'investment_id'], axis=1).values
    data = torch.tensor(data).float()
    with torch.no_grad():
        preds = model.forward(data, [])
    sub_df['target'] = preds.view(-1).cpu().numpy()
    env.predict(sub_df) 

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
